this script crosscheck the country of the funding institution with the institutions from this paper https://academic.oup.com/jncics/article/3/4/pkz069/5582653

In [11]:
import pandas as pd
import re

In [48]:
inst_lili_path="../../data/nf_folder/Data/RawData/Funder_Count.txt"
inst_lili=pd.read_csv(inst_lili_path,sep=";",
                      error_bad_lines=False,warn_bad_lines=False)

In [49]:
inst_lili=inst_lili.reset_index()
inst_lili.drop(columns='count',inplace=True)
inst_lili.columns=['inst','cntry','count']

In [10]:
inst_lili.head()

,org,cntry,count
0,NPRP grant from the Qatar National Research Fund,QATAR,125
1,Innovative Research Group of Hubei Province,PEOPLES-R-CHINA,68
2,RPB Physician-Scientist Award,USA,12
3,OEMT,PEOPLES-R-CHINA,3
4,NICUS,SPAIN,4


In [19]:
inst_cancer_path='../../data/nf_folder/Data/RawData/institution_validity/Funding_Cancer.docx'

In [20]:
import docx2txt
text = docx2txt.process(inst_cancer_path)

In [22]:
type(text)

str

In [43]:
rlist=[]
for ss in text.split("\n"):
    if len(ss)>0 and "," in ss:
        s=ss.split(",")
        cntry=s[-1].strip(" ")
        inst=",".join(s[:-1]).strip(' ')
        rlist.append([inst,cntry])

In [57]:
cancer_df=pd.DataFrame(rlist,columns=['inst_raw','cntry'])

In [12]:
cancer_df.shape

(4737, 2)

In [58]:
cancer_df['inst']=cancer_df['inst_raw'].apply(lambda x:x.split('-')[0].strip(" "))

In [59]:
inst_meta=cancer_df.merge(inst_lili,on='inst',how='left')

In [60]:
inst_meta=inst_meta.rename(columns={'cntry_x':'cntry_cancer','cntry_y':'cntry_lili'})

In [64]:
inst_meta=inst_meta[['inst_raw','inst','cntry_cancer','cntry_lili','count']]
inst_meta = inst_meta.sort_values(by='cntry_lili')

In [67]:
inst_meta.to_excel('inst_checkding_v3.xlsx')

In [2]:
inst_ff_path="../../data/nf_folder/Data/RawData/institution_validity/funder_country_all.xlsx"
inst_ff=pd.read_excel(inst_ff_path)

In [3]:
inst_meta=pd.read_excel('inst_checkding_v3.xlsx')

In [4]:
inst_meta.head()

,Unnamed: 0,inst_raw,inst,cntry_cancer,cntry_lili,count,label
0,3047,Shire,Shire,Ireland,ENGLAND,2600.0,0.0
1,3071,Hutchison Whampoa Limited,Hutchison Whampoa Limited,Japan,ENGLAND,156.0,0.0
2,1976,Cridlan Fund,Cridlan Fund,USA,ENGLAND,11.0,0.0
3,2198,Jacqueline Seroussi Memorial Foundation for Ca...,Jacqueline Seroussi Memorial Foundation for Ca...,USA,ENGLAND,15.0,0.0
4,3659,Transgene,Transgene,USA,FRANCE,31.0,0.0


In [5]:
inst_ff=inst_ff[['Organisation','labela']]
inst_ff.columns=['inst','proc']

In [6]:
inst_meta=inst_meta.merge(inst_ff,on='inst',how='left')

In [7]:
inst_meta.to_excel('inst_checking_v3.xlsx',index=False)